### Topic Analysis using LDA Topic Model (Community Dataset #2: 15/12/2020 - 31/01/2021)

#### Import core libraries

In [1]:
# Import Core library
import sys
import re, numpy as np, pandas as pd, matplotlib.pyplot as plt, nltk, datetime as dt, seaborn as sns
import emoji
import little_mallet_wrapper
import plotly.express as px
import wordcloud
import openpyxl
from pprint import pprint
from cleantext import clean

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt

# NLTK Stop words
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
[nltk_data] Downloading package stopwords to C:\Users\Ivory-
[nltk_data]     UALR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Import Dataset

In [2]:
# Import Dataset
df = pd.read_excel('../Community306_raw.xlsx')

In [ ]:
# view column titles of dataset
df.columns

In [3]:
df['create_year'] = pd.to_datetime(df.created_at, format='%d/%m/%Y', errors='coerce')
df['year'] = pd.to_datetime(df.created_at).dt.strftime('%Y')
df['month'] = pd.to_datetime(df.created_at).dt.strftime('%m')

start_date = "12/15/2020"
end_date = "01/31/2021"
after_start_date = df['create_year'] > start_date
before_end_date = df['create_year'] < end_date
between_two_dates = after_start_date & before_end_date

stream_two = df.loc[between_two_dates]

In [9]:
# stream_two.to_excel(r"C:\Users\Ivory-UALR\Documents\COSMOS\Billy's Project\community_topic_model\2nd_community_dataset\2nd_analysis_LDAmodel\stream_two.xlsx")

In [6]:
stream_two.head()

,Unnamed: 0,id,source_id,created_at,geo_source,truncated,text,text_sentiment,text_toxicity,lang,...,in_reply_to_status_id_str,in_reply_to_user_id_str,in_reply_to_screen_name,entities,possibly_sensitive,coordinates,place,create_year,year,month
0,1,1.346170e+18,18,2021-01-04 19:01:23,1,0,@HHere4trump @sbacon0410 #stopthesteal #stopth...,None,None,und,...,1346156649287987201,None,None,"{""entities"": {""hashtags"": [{""end"": 38, ""tag"": ...",0,None,None,2021-01-04 19:01:23,2021,01
1,3,1.345954e+18,18,2021-01-04 04:45:23,None,0,@RudyGiuliani @CaliConserv1 #StoptheSteal #Sto...,None,None,und,...,1345924505483546624,None,None,"{""entities"": {""hashtags"": [{""end"": 41, ""tag"": ...",0,None,None,2021-01-04 04:45:23,2021,01
2,4,1.345954e+18,18,2021-01-04 04:45:13,None,0,#StoptheSteal #StoptheSteal #StoptheSteal #Sto...,None,None,und,...,None,None,None,"{""entities"": {""urls"": [{""end"": 303, ""url"": ""ht...",0,None,None,2021-01-04 04:45:13,2021,01
3,5,1.345952e+18,18,2021-01-04 04:34:34,None,0,#StoptheSteal #StoptheSteal #StoptheSteal #Sto...,None,None,und,...,None,None,None,"{""entities"": {""urls"": [{""end"": 303, ""url"": ""ht...",0,None,None,2021-01-04 04:34:34,2021,01
4,6,1.345825e+18,18,2021-01-03 20:12:15,None,0,@realDonaldTrump @4Libertyinlaw #StoptheSteal ...,None,None,und,...,1345798202650460162,None,None,"{""entities"": {""hashtags"": [{""end"": 45, ""tag"": ...",0,None,None,2021-01-03 20:12:15,2021,01


#### Process Dataset

In [7]:
# Function to remove twitter specific characters
def get_url_patern():
    return re.compile(
        r'(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))'
        r'[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})')

def get_hashtags_pattern():
    return re.compile(r'#\w*')

def get_single_letter_words_pattern():
    return re.compile(r'(?<![\w\-])\w(?![\w\-])')

def get_blank_spaces_pattern():
    return re.compile(r'\s{2,}|\t')

def get_twitter_reserved_words_pattern():
    return re.compile(r'(RT|rt|FAV|fav|VIA|via)')

def get_mentions_pattern():
    return re.compile(r'@\w*')

def process_text(word):
  word=re.sub(pattern=get_url_patern(), repl="", string=word)
  word=re.sub(pattern=get_mentions_pattern(), repl="", string=word)
  word=re.sub(pattern=get_hashtags_pattern(), repl="", string=word)
  word=re.sub(pattern=get_twitter_reserved_words_pattern(), repl='', string=word)
  word=re.sub(r'http\S+', "", word)  # remove http links
  word=re.sub(r'bit.ly/\S+', "", word)  # rempve bitly links
  word=word.strip('[link]')  # remove [links]
  word=re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', "", word)  # remove retweet
  word=re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', "", word)  # remove tweeted at
  word=word.encode('ascii', 'ignore').decode('ascii')
#   word=clean(word, no_emoji=True)
  return word

<>:30: DeprecationWarning: invalid escape sequence \s
<>:30: DeprecationWarning: invalid escape sequence \s
C:\Users\IVORY-~1\AppData\Local\Temp/ipykernel_25264/758640023.py:30: DeprecationWarning: invalid escape sequence \s
  word=re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', "", word)  # remove retweet


In [15]:
# Remove twitter specific characters
stream_two.text = stream_two.apply(lambda row: process_text(row.text), 1)

c:\Users\Ivory-UALR\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [18]:
# Remove punctuations, stop words, digits and turn text to lowercase
training_data = [little_mallet_wrapper.process_string(text, numbers='remove') for text in stream_two.text]
data = training_data
data[:5]

['', '', '', '', '']

In [19]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:20])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['dem', 'stop', 'screaming'], ['fail', 'mention'], ['stay', 'away', 'wed', 'jan', 'let', 'trump', 'brownshis', 'violent', 'protest', 'thing', 'alone']]


#### N-grams & Lemmatization

In [20]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

[]


In [21]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [22]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.max_length = 40000000

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[[]]


In [23]:
# Export lemmatized data for document similarity evaluation
corpus_df = pd.DataFrame(data_lemmatized)
corpus_df.to_csv('master_corpus(51-04)_duplicate.csv', index=False, header=False)

In [19]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:20])

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [(0, 1), (1, 1)], [(2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


#### Building the LDA Model

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=9, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [21]:
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=texts, start=2, limit=40, step=2)

# # Show graph
# limit=40; start=2; step=2;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [22]:
# Print the coherence scores
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
    
# From the pyLDAvis visualization, the optimal number of topics is 6

#### Visualize Topic Distribution & Keywords

In [23]:
topic_dist = pd.DataFrame()

# Code to view the Ist 10 keywords per topic
for index, topic in lda_model.show_topics(formatted=False, num_words= 10):
    words = [w[0] for w in topic]
    topic_distributions_df = ["".join(topic[:4]) for topic in words]
    topic_dist = topic_dist.append(pd.Series([index, words]), ignore_index=True)
    
    # print('Topic: {} \nWords: {}'.format(index, words))
    # print(topic_distributions_df)
    
topic_dist.columns = ['Topic_Number', 'Topic_Words']
topic_dist

# Export topic word distribution per topic to excel file
topic_dist.to_excel(r"C:\Users\Ivory-UALR\Documents\COSMOS\Billy's Project\community_topic_model\2nd_community_dataset\2nd_analysis_LDAmodel\2nd_analysis_LDA_topic_words.xlsx", index=False, header=True)

In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

C:\Users\Ivory-UALR\AppData\Local\Programs\Python\Python39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.080118  0.010853       1        1  20.915449
5     -0.159052 -0.332138       2        1  18.085997
7     -0.185318  0.226341       3        1  18.018714
1     -0.209631  0.191180       4        1  15.648652
8     -0.093619 -0.170473       5        1  14.265830
4      0.145830  0.021242       6        1   5.272341
2      0.165367  0.000612       7        1   4.641517
3      0.212255  0.025714       8        1   1.913658
0      0.204286  0.026670       9        1   1.237842, topic_info=            Term         Freq        Total Category  logprob  loglift
199        stand  9274.000000  9274.000000  Default  30.0000  30.0000
14      election  8886.000000  8886.000000  Default  29.0000  29.0000
21          vote  7787.000000  7787.000000  Default  28.0000  28.0000
138         take  6560.000000  6560.000000  Default  27.0000  27.0000
789      elector  6705.000000  6705.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
2103        flag    31.053655    31.996563   Topic9  -4.9809   4.3619
2283       stick    28.552863    29.496436   Topic9  -5.0648   4.3593
1374  infiltrate    27.223673    28.166524   Topic9  -5.1125   4.3578
4600    customer    59.478593    61.617796   Topic9  -4.3310   4.3565
5350     convene    38.290206    41.405503   Topic9  -4.7714   4.3136

[319 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
213       3  0.999619     accept
368       3  0.999347     action
905       2  0.998532    address
163       3  0.998458      agree
452       3  0.998995      allow
...     ...       ...        ...
305       1  0.999140        win
98        1  0.997964       work
254       2  0.998277      world
429       1  0.999148       year
1621      6  0.992979  yesterday

[349 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 6, 8, 2, 9, 5, 3, 4, 1])

In [25]:
# Export LDA Model
pyLDAvis.save_html(vis, r"C:\Users\Ivory-UALR\Documents\COSMOS\Billy's Project\community_topic_model\2nd_community_dataset\2nd_analysis_LDAmodel\2nd_analysis_lda_vis.html")

#### Visualize Topic Distribution (Dominant topic per text, Topic distribution over time (Topic Stream))

In [26]:
# View dominant topic per document (tweet) and topic percent contribution per tweet
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
1,1,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
2,2,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
3,3,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
4,4,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
5,5,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
6,6,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
7,7,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
8,8,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",
9,9,6.0,0.2118,"never, count, patriot, get, thank, need, see, ...",


In [27]:
# Add year column to topic distribution dataset
dates = pd.Series(stream_two['create_year'])
topic_dist = pd.concat([df_dominant_topic, dates], axis=1)
# Drop NaN rows
topic_dist.dropna()

# Group Dataframe by year and aggregate by count()
topic_dist['create_year'] = pd.to_datetime(topic_dist['create_year'])
topic_dist_count = topic_dist.groupby(['Dominant_Topic', pd.Grouper(key='create_year', freq='D')])['Dominant_Topic'].agg(['count']).reset_index()
print(topic_dist_count)

# return a sorted DataFrame by date then count
topic_dist_count = topic_dist_count.sort_values(by=['create_year', 'count'])
# reset the index
topic_dist_count = topic_dist_count.reset_index(drop=True)

     Dominant_Topic create_year  count
0               0.0  2021-01-03      1
1               0.0  2021-01-04      1
2               0.0  2021-01-06      1
3               0.0  2021-01-08      1
4               1.0  2021-01-01    174
..              ...         ...    ...
203             8.0  2021-01-26      3
204             8.0  2021-01-27     11
205             8.0  2021-01-28     26
206             8.0  2021-01-29      8
207             8.0  2021-01-30      9

[208 rows x 3 columns]


In [28]:
# Plot topic distribution over time
topic_stream = px.area(topic_dist_count, x='create_year', y='count', color='Dominant_Topic')
topic_stream


In [29]:
# Export topic stream plot
topic_stream.write_html(r"C:\Users\Ivory-UALR\Documents\COSMOS\Billy's Project\community_topic_model\2nd_community_dataset\2nd_analysis_LDAmodel\2nd_analysis_topicstream_vis.html")

#### Visualizing topic streams by weeks

In [32]:
# Add year column to topic_dist_week distribution dataset
dates = pd.Series(stream_two['create_year'])
topic_dist_week = pd.concat([df_dominant_topic, dates], axis=1)

# Drop NaN rows
topic_dist_week = topic_dist_week.dropna()

# Add a year column to the topic_dist_week dataframe
topic_dist_week['year'] = pd.to_datetime(topic_dist_week.create_year).dt.strftime('%Y')

# Add a week_number column to the topic_dist_week dataframe
topic_dist_week['week_number'] = topic_dist_week.create_year.apply(lambda x: x.isocalendar()[1])

# Convert weeks to date_range and add date_range column to topic_dist_week dataframe
def date_range(year, week): # Function to return date range per week
    monday = dt.datetime.strptime(f'{year}-{week}-1', "%Y-%W-%w").date()
    return (monday.strftime("%m-%d-%Y"), (monday + dt.timedelta(days=6.9)).strftime("%m-%d-%Y"))

for column in topic_dist_week:
    date_range_list = []
    years = topic_dist_week['year'].values
    weeks = topic_dist_week['week_number'].values
    for (year, week) in zip(years, weeks):
        date_interval = date_range(year, week)
        date_range_list.append(date_interval)

topic_dist_week['date_range'] = pd.Series(date_range_list)

In [33]:
# Generate week-date_interval information
years = topic_dist_week['year'].values
weeks = topic_dist_week['week_number'].values
for (year, week) in zip(years, weeks):
        date_interval = date_range(year, week)
        date_range_list.append(date_interval)

unique_weeks = pd.Series(list(set(weeks)))
unique_interval = pd.Series(list(set(date_range_list)))

topic_week_agg = pd.DataFrame()
topic_week_agg['week_number'] = unique_weeks
topic_week_agg['date_interval'] = unique_interval
topic_week_agg

,week_number,date_interval
0,1,"(01-18-2021, 01-24-2021)"
1,2,"(01-25-2021, 01-31-2021)"
2,3,"(01-03-2022, 01-09-2022)"
3,4,"(01-11-2021, 01-17-2021)"
4,53,"(01-04-2021, 01-10-2021)"


In [34]:
# Export week information
topic_week_agg.to_excel(r"C:\Users\Ivory-UALR\Documents\COSMOS\Billy's Project\community_topic_model\2nd_community_dataset\2nd_analysis_LDAmodel\2nd_analysis_week_information.xlsx")

#### Plot topics by weeks

In [35]:
# Group Dataframe by week_number and aggregrate by count() NB: Convert datee_range list to date_range tuple
topic_dist_week_count = topic_dist_week.groupby(['Dominant_Topic', pd.Grouper(key='week_number')])['Dominant_Topic'].agg(['count']).reset_index()
# return a sorted DataFrame by date then count
topic_dist_week_count = topic_dist_week_count.sort_values(by=['week_number', 'count'])
# reset the index
topic_dist_week_count = topic_dist_week_count.reset_index(drop=True)
topic_dist_week_count

,Dominant_Topic,week_number,count
0,0.0,1,3
1,3.0,1,26
2,2.0,1,104
3,4.0,1,109
4,8.0,1,1814
5,7.0,1,3318
6,1.0,1,3880
7,5.0,1,3911
8,6.0,1,13724
9,2.0,2,2


In [36]:
# Plot topic distribution over time
topic_stream_week = px.area(topic_dist_week_count, x='week_number', y='count', color='Dominant_Topic')
topic_stream_week

In [37]:
# Export topic stream plot
topic_stream_week.write_html(r"C:\Users\Ivory-UALR\Documents\COSMOS\Billy's Project\community_topic_model\2nd_community_dataset\2nd_analysis_LDAmodel\2nd_analysis_topicstream_vis(weeks).html")